<a href="https://colab.research.google.com/github/dalatorrem/Diplomado_Metodos_UCentral_2023/blob/main/Lectura_33_Confirm_FA_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<p align="center">
  <img src="https://raw.githubusercontent.com/lacamposm/Diplomado_Metodos_UCentral/main/data/images/imagen_ucentral.jpg" alt="logo_Ucentral" width="400px" height="300px">
</p>

## ***Protocolo de módulos y funciones.***

In [1]:
%%capture
!pip install factor_analyzer

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity
## class para realizar CFA.
from factor_analyzer import ConfirmatoryFactorAnalyzer, ModelSpecificationParser
##
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
def view_loadings(fa, df):
  loadings, n_factors = fa.loadings_, fa.get_params()["n_factors"]
  columns = [f"factor{i}" for i in range(1,n_factors+1)]
  df_loadings = pd.DataFrame(loadings, index = df.columns, columns = columns)
  return df_loadings

In [4]:
def get_communa_uniquess(fa, df_fa):
  communa, uniqueness  = fa.get_communalities(), fa.get_uniquenesses()
  temp1 = pd.DataFrame(communa, columns=["comunalidades"], index = df_fa.columns).T
  temp2 = pd.DataFrame(uniqueness, columns=["especificidad"], index = df_fa.columns).T
  df = pd.concat([temp1,temp2])
  return df

In [5]:
def get_statistics_cfa(cfa):
  """
      Función que encuentra algunas estadísticas de un CFA (Confirmatory Factor Analysis)
      Input: Objeto CFA from factor_Analyzer packet.
      Output: pd.DataFrame con las medidad GFI, RMSRA, AFI,BIC, AIC del ajuste por CFA.
  """
  ### Auxiliar Function.
  from math import factorial
  def comb(a,b):
    comb = factorial(a)/(factorial(b)*factorial(a-b))
    return comb
  ###
  S, inverse_sigma = cfa.cov_, np.linalg.inv(cfa.get_model_implied_cov())
  BIC, AIC, n_factors = cfa.bic_, cfa.aic_, cfa.factor_varcovs_.shape[0]
  n = S.shape[0]
  k = n + n*(n-1)/2                             ## Valores no repetidos en la matriz varianzas-covarianzas
  params = 2*n+comb(n_factors,2)                ## Parámetros a estimar
  df = k - params                               ## Grados de libertad.
  ## Cálculo GFI (goodness-of-fit index)
  temp = S@inverse_sigma - np.identity(n)
  trace1 = np.trace(temp@temp)
  trace2 = np.trace(S@inverse_sigma@S@inverse_sigma)
  GFI = 1-(trace1/trace2)
  ## Cálculo AGFI.
  AGFI = 1 - (k/df)*(1 - GFI)
  ### Cálculo de RMSRA
  RMSRA = np.sqrt(sum(((S- cfa.get_model_implied_cov()).reshape(-1))**2)/(n**2))
  ## Retornando la info
  stats_df = pd.DataFrame([GFI,AGFI,RMSRA,BIC,AIC], index = ["GFI","AGFI","RMSRA","BIC","AIC"],
                          columns = ["value"]).T
  return stats_df

## ***Introducción***

El análisis factorial exploratorio se utiliza en los primeros pasos de la investigación de un conjunto de datos multivariados para determinar si el modelo de análisis factorial es útil para proporcionar una manera parsimoniosa de describir y teniendo en cuenta las relaciones entre las variables observadas. El analisis
determinará qué variables observadas están más altamente correlacionadas con los ***factores comunes*** y cuántos factores comunes se necesitan para dar una descripción adecuada de los datos. En un análisis factorial exploratorio, sin restricciones se colocan en qué ***variables manifiestas*** se cargan en qué factores.

En esta breve lectura vamos a considerar modelos de análisis factorial confirmatorio en los que se permite que ***variables manifiestas particulares*** se relacionen con factores particulares mientras que otras variables manifiestas están restringidas para tener cargas cero en algunos de los factores.

Un modelo de análisis factorial confirmatorio puede surgir de consideraciones teóricas o basarse en los resultados de un análisis factorial exploratorio donde el investigador podría desear postular un modelo específico para un nuevo conjunto de datos similares, uno en que las cargas de algunas variables sobre algunos factores se fijan en cero porque eran "pequeños" en el análisis exploratorio. Es importante enfatizar que
mientras que es perfectamente apropiado llegar a un modelo factorial para someterlo a un análisis confirmatorio de un análisis factorial exploratorio, el modelo debe ser probado en un nuevo conjunto de datos. Los modelos no deben ser generados y probados en la misma data.

Los modelos de ***análisis factorial confirmatorio*** son un subconjunto de un enfoque más general para modelar variables latentes conocido como [***modelo de ecuaciones estructurales***](https://es.wikipedia.org/wiki/Ecuaciones_estructurales) o modelado de ***estructura de covarianza***. Dichos modelos permiten tanto la respuesta como las variables latentes explicativas vinculadas por una serie de ecuaciones lineales. A pesar de que es más complejo que los modelos de análisis factorial confirmatorio, el objetivo de
modelos de ecuaciones es esencialmente lo mismo, es decir, para explicar las correlaciones o covarianzas de las variables observadas en términos de las relaciones de estas variables a las supuestas ***variables latentes subyacentes*** y las relaciones postuladas entre las propias ***variables latentes***.



## ***Estimación, identificación y evaluación de la idoneidad para modelos de factores confirmatorios y de ecuaciones estructurales.***



Los modelos de ecuaciones estructurales contendrán un número de parámetros que se necesitan estimar a partir de la matriz de covarianza o correlación de las variables manifiestas, la estimación implica encontrar valores para los parámetros del modelo que minimizan una función de discrepancia que indica la magnitud de las diferencias entre los elementos de $S$, la matriz de covarianza observada de las variables manifiestas y las de $Σ(θ)$, la matriz de covarianza implícita en el modelo ajustado (es decir, una matriz cuyos elementos son funciones de los parámetros del modelo), contenido en el vector $θ = (θ_1, . . . , θ_t)^{t}$.

El método más comúnmente utilizado para estimar los parámetros en los modelos de factores confirmatorios y de ecuaciones estructurales es la máxima verosimilitud bajo la suposición de que los datos observados tienen una distribución normal multivariante. Debemos minimizar la función de discrepancia, $FML$, dada por:

$$FML(S, Σ(θ)) = log |Σ(θ)| − log |S| + tr(SΣ(θ)^{-1}) − q$$

Vemos que al variar los parámetros $θ_1, ...,θ_t$ de modo que $Σ(θ)$ se vuelve más como $S$, $FML$ se vuelve
menor.

## ***Evaluación del ajuste de un modelo.***

Una vez que se ha identificado un modelo y se han estimado sus parámetros, el siguiente paso es evaluar qué tan bien se ajusta la matriz de covarianza predicha por el modelo a la matriz de covarianza de las variables manifiestas. Una medida global de ajuste de un modelo es proporcionada por la estadística de razón de verosimilitud dada por $X_2=(N − 1)FMLmin$, donde N es el tamaño de la muestra y $FMLmin$ es el valor minimizado de la función de discrepancia de máxima verosimilitud dada anteriormente.


Si el tamaño de la muestra es lo suficientemente grande, la estadística $X_2$ proporciona una prueba de que la matriz de covarianza de la población de las ***variables manifiestas*** es igual a la covarianza implícita del modelo ajustado frente a la hipótesis alterna de que la matriz de población no tiene restricciones. Bajo la igualdad hipótesis, $X_2$ tiene una distribución chi-cuadrado con grados de libertad $ν$ $\frac{1}{2}q(q+1)-t$, donde $t$ es el número de parámetros libres en el modelo.


La estadística de razón de verosimilitud es a menudo la única medida de ajuste citada para un modelo ajustado, pero por sí solo tiene un uso práctico limitado porque en grandes muestras incluso desviaciones relativamente triviales de la hipótesis nula de igualdad dará lugar a su rechazo. En consecuencia, en muestras grandes, la mayoría de los modelos pueden ser rechazado como estadísticamente insostenible. Una forma más satisfactoria de utilizar la prueba es para una comparación de una serie de modelos anidados donde una gran diferencia en la estadística para dos modelos en comparación con la diferencia en los grados de libertad de los modelos indica que los parámetros adicionales en uno de los modelos proporcionan una mejora genuina en el ajuste.

Surgen problemas adicionales con la estadística de razón de verosimilitud cuando las observaciones provienen de una población donde las variables manifiestas tienen un valor distribución no normal. Se puede demostrar que en el caso de una distribución con curtosis sustancial, la distribución de chi-cuadrado puede ser una mala aproximación para la distribución nula de $X_2$.

Quizás la mejor manera de evaluar el ajuste de un modelo es usar el estadístico $X_2$ junto con uno o más de los siguientes procedimientos:

1. Inspección visual de las covarianzas residuales (es decir, las diferencias entre las covarianzas de las variables manifiestas y las predichas por el ajustado
modelo). Estos residuos deben ser pequeños en comparación con los valores de las covarianzas o correlaciones observadas.

2. Examinar los errores estándar de los parámetros y las correlaciones entre estas estimaciones. Si las correlaciones son grandes, puede indicar que
el modelo que se está ajustando es casi desconocido.

3. Valores de parámetros estimados fuera de su rango posible; es decir, las varianzas negativas o los valores absolutos de las correlaciones mayores que la unidad son a menudo una indicación de que el modelo ajustado es fundamentalmente incorrecto para los datos.


Además, se han sugerido una serie de índices de ajuste que a veces pueden ser útiles.

- ***GFI (índice de bondad de ajuste)***:

Se basa en la relación de la suma de las distancias al cuadrado entre las matrices observadas y las reproducida por la covarianza del modelo, lo que permite la escala. $GFI$ mide la cantidad de varianza y covarianza en $S$ que se explica por la matriz de covarianza predicha por el modelo, es decir $Σ(θ)$. $GFI$ puede tomar valores entre cero (sin ajuste) y uno (ajuste perfecto); en la practica, sólo los valores por encima de 0,9 o incluso 0,95 sugieren un nivel de ajuste aceptable.

- ***AGFI (índice de bondad de ajuste ajustado):***

Este ajusta el índice $GFI$ para el grados de libertad de un modelo en relación con el número de variables. $AGFI$ se pueden utilizar para comparar el ajuste de dos modelos diferentes con los mismos datos o comparar el ajuste de modelos con diferentes datos, por ejemplo conjuntos de datos masculinos y femeninos.

-  ***RMSR (índice de ajuste es el residuo de la raíz cuadrada media):***

Es la raíz cuadrada de las diferencias medias cuadráticas entre los elementos en $S$ y $Σ$.
Generalmente se considera que un valor de $RMSR<0.05$ indica un ajuste razonable.

## ***Metodología.***

En un modelo factorial confirmatorio las cargas para algunas variables observadas en algunos de los factores comunes postulados se fijarán a priori en cero. Además, algunas correlaciones entre factores también pueden fijarse en cero. El modelo se ajusta a un conjunto de datos estimando sus parámetros libres; es decir, los que no fijado en cero por el investigador. La estimación suele ser por máxima verosimilitud utilizando la función de discrepancia $FML$.

### ***Ejemplo 1.***



In [6]:
url_base = "https://raw.githubusercontent.com/lacamposm/Metodos_Estadisticos/main/data/"
df = pd.read_csv(url_base + "CFA_dataset.csv", index_col = 0)
print(df.shape)
df = df.dropna()
df.head()

(2800, 25)


,A1,A2,A3,A4,A5,C1,C2,C3,C4,C5,E1,E2,E3,E4,E5,N1,N2,N3,N4,N5,O1,O2,O3,O4,O5
61617,2.0,4.0,3.0,4.0,4.0,2.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,4.0,2.0,2.0,3.0,3.0,6,3.0,4.0,3.0
61618,2.0,4.0,5.0,2.0,5.0,5.0,4.0,4.0,3.0,4.0,1.0,1.0,6.0,4.0,3.0,3.0,3.0,3.0,5.0,5.0,4.0,2,4.0,3.0,3.0
61620,5.0,4.0,5.0,4.0,4.0,4.0,5.0,4.0,2.0,5.0,2.0,4.0,4.0,4.0,5.0,4.0,5.0,4.0,2.0,3.0,4.0,2,5.0,5.0,2.0
61621,4.0,4.0,6.0,5.0,5.0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,4.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,3.0,3,4.0,3.0,5.0
61622,2.0,3.0,3.0,4.0,5.0,4.0,4.0,5.0,3.0,2.0,2.0,2.0,5.0,4.0,5.0,2.0,3.0,4.0,4.0,3.0,3.0,3,4.0,3.0,3.0


In [7]:
## Split data:
## 1. X_EFA para el análisis Exploratorio.
## 2. X_EFA para el análisis Confirmatorio.
X_EFA, X_CFA = train_test_split(df, test_size = 0.5, random_state = 0)

In [8]:
X_EFA.describe().T

,count,mean,std,min,25%,50%,75%,max
A1,1218.0,2.380952,1.389962,1.0,1.0,2.0,3.0,6.0
A2,1218.0,4.777504,1.173770,1.0,4.0,5.0,6.0,6.0
A3,1218.0,4.579639,1.325056,1.0,4.0,5.0,6.0,6.0
A4,1218.0,4.633005,1.491954,1.0,4.0,5.0,6.0,6.0
A5,1218.0,4.501642,1.315009,1.0,4.0,5.0,5.0,6.0
C1,1218.0,4.490969,1.267248,1.0,4.0,5.0,5.0,6.0
C2,1218.0,4.335796,1.332132,1.0,4.0,5.0,5.0,6.0
C3,1218.0,4.266010,1.287004,1.0,4.0,4.0,5.0,6.0
C4,1218.0,2.577176,1.370917,1.0,1.0,2.0,4.0,6.0
C5,1218.0,3.296388,1.628627,1.0,2.0,4.0,5.0,6.0


In [9]:
## No hay problema con Barlett.
calculate_bartlett_sphericity(X_EFA)

(9364.130301741801, 0.0)

In [10]:
## KMO. No hay problema con KMO. Viable Factor Analyze.
calculate_kmo(X_EFA)[1]

0.8399488514029732

In [11]:
## Análisis facotorial.
scaler_X_EFA = StandardScaler()
scaler_X_EFA.fit(X_EFA)
scaled_data_X_EFA = scaler_X_EFA.transform(X_EFA)
###
fa = FactorAnalyzer(n_factors=5, method = "ml", rotation = "varimax")
fa.fit(scaled_data_X_EFA)

FactorAnalyzer(method='ml', n_factors=5, rotation='varimax', rotation_kwargs={})

In [12]:
loadings = view_loadings(fa,X_EFA)

In [13]:
loadings.sort_values(by = ["factor1"], ascending = False)

,factor1,factor2,factor3,factor4,factor5
N1,0.829128,0.088414,-0.194380,-0.023096,-0.079655
N2,0.797892,0.047519,-0.170719,-0.035355,-0.039606
N3,0.737090,-0.093718,-0.021142,-0.073982,0.041584
N4,0.564604,-0.379103,-0.025193,-0.158820,0.053312
N5,0.480814,-0.186069,0.098538,-0.028508,-0.152395
C5,0.223412,-0.186143,-0.043592,-0.580535,0.032702
C4,0.208782,-0.083431,0.005370,-0.641957,-0.096509
E2,0.200247,-0.683278,-0.180634,-0.090904,-0.078008
O4,0.185714,-0.257020,0.145823,-0.030770,0.366043
O2,0.172001,-0.000551,0.097433,-0.132257,-0.481200


In [14]:
abs(loadings).sort_values(by = ["factor2"], ascending = False)

,factor1,factor2,factor3,factor4,factor5
E2,0.200247,0.683278,0.180634,0.090904,0.078008
E4,0.097088,0.607296,0.363755,0.098592,0.010262
E1,0.037639,0.602626,0.138991,0.002542,0.074567
E5,0.047946,0.487704,0.131193,0.316583,0.243273
E3,0.005461,0.466919,0.347878,0.088451,0.337863
N4,0.564604,0.379103,0.025193,0.158820,0.053312
A5,0.129239,0.339381,0.571597,0.073708,0.097621
O3,0.016136,0.262937,0.148804,0.101602,0.638304
O4,0.185714,0.257020,0.145823,0.030770,0.366043
A3,0.018005,0.249571,0.673201,0.086527,0.086987


In [15]:
abs(loadings).sort_values(by = ["factor3"], ascending = False)

,factor1,factor2,factor3,factor4,factor5
A3,0.018005,0.249571,0.673201,0.086527,0.086987
A2,0.033662,0.158358,0.634297,0.118708,0.068411
A5,0.129239,0.339381,0.571597,0.073708,0.097621
A4,0.014885,0.179487,0.441037,0.228627,0.115739
A1,0.148304,0.036302,0.435059,0.070646,0.020126
E4,0.097088,0.607296,0.363755,0.098592,0.010262
E3,0.005461,0.466919,0.347878,0.088451,0.337863
N1,0.829128,0.088414,0.194380,0.023096,0.079655
E2,0.200247,0.683278,0.180634,0.090904,0.078008
N2,0.797892,0.047519,0.170719,0.035355,0.039606


In [16]:
abs(loadings).sort_values(by = ["factor4"], ascending = False)

,factor1,factor2,factor3,factor4,factor5
C2,0.082716,0.018330,0.067005,0.649033,0.143836
C4,0.208782,0.083431,0.005370,0.641957,0.096509
C5,0.223412,0.186143,0.043592,0.580535,0.032702
C1,0.004399,0.045291,0.026743,0.551665,0.219123
C3,0.003083,0.017645,0.130314,0.532995,0.027233
E5,0.047946,0.487704,0.131193,0.316583,0.243273
A4,0.014885,0.179487,0.441037,0.228627,0.115739
N4,0.564604,0.379103,0.025193,0.158820,0.053312
O2,0.172001,0.000551,0.097433,0.132257,0.481200
A2,0.033662,0.158358,0.634297,0.118708,0.068411


In [17]:
abs(loadings).sort_values(by = ["factor5"], ascending = False)

,factor1,factor2,factor3,factor4,factor5
O3,0.016136,0.262937,0.148804,0.101602,0.638304
O1,0.002126,0.184498,0.099371,0.111461,0.562826
O5,0.074232,0.015543,0.033271,0.078539,0.512396
O2,0.172001,0.000551,0.097433,0.132257,0.481200
O4,0.185714,0.257020,0.145823,0.030770,0.366043
E3,0.005461,0.466919,0.347878,0.088451,0.337863
E5,0.047946,0.487704,0.131193,0.316583,0.243273
C1,0.004399,0.045291,0.026743,0.551665,0.219123
N5,0.480814,0.186069,0.098538,0.028508,0.152395
C2,0.082716,0.018330,0.067005,0.649033,0.143836


In [18]:
## comunalidades
get_communa_uniquess(fa,X_EFA).round(4).T.sort_values(by=["especificidad"], ascending = False).T

,A1,O4,O5,O2,A4,N5,C3,C1,O1,E1,E5,C5,A2,C2,E3,C4,A5,N4,O3,E4,A3,E2,N3,N2,N1
comunalidades,0.218,0.2567,0.2756,0.2881,0.2926,0.2996,0.3021,0.3551,0.3731,0.3895,0.4168,0.4246,0.4473,0.4536,0.461,0.472,0.4736,0.4912,0.5093,0.5204,0.5309,0.5539,0.5597,0.6709,0.7399
especificidad,0.782,0.7433,0.7244,0.7119,0.7074,0.7004,0.6979,0.6449,0.6269,0.6105,0.5832,0.5754,0.5527,0.5464,0.539,0.528,0.5264,0.5088,0.4907,0.4796,0.4691,0.4461,0.4403,0.3291,0.2601


## ***CFA***

In [19]:
scaler_X_CFA = StandardScaler()
scaler_X_CFA.fit(X_CFA)
scaled_data_X_CFA = scaler_X_CFA.transform(X_CFA)

In [20]:
## Análisis factorial confirmatorio.
model_dict = {"F1":[f"A{i}" for i in range(1,6)], "F2":[f"C{i}" for i in range(1,6)],
              "F3":[f"E{i}" for i in range(1,6)], "F4":[f"N{i}" for i in range(1,6)],
              "F5":[f"O{i}" for i in range(1,6)]}
model_dict

{'F1': ['A1', 'A2', 'A3', 'A4', 'A5'],
 'F2': ['C1', 'C2', 'C3', 'C4', 'C5'],
 'F3': ['E1', 'E2', 'E3', 'E4', 'E5'],
 'F4': ['N1', 'N2', 'N3', 'N4', 'N5'],
 'F5': ['O1', 'O2', 'O3', 'O4', 'O5']}

In [21]:
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(scaled_data_X_CFA, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp = True)

In [22]:
## CFA.
cfa.fit(scaled_data_X_CFA)

ConfirmatoryFactorAnalyzer(n_obs=1218,
                           specification=<factor_analyzer.confirmatory_factor_analyzer.ModelSpecification object at 0x786d3fdd3850>)

In [23]:
get_statistics_cfa(cfa)

,GFI,AGFI,RMSRA,BIC,AIC
value,0.850817,0.81704,0.078992,80918.077544,79948.134109


In [24]:
cfa.loadings_

array([[-0.32337308,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.64926483,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.7500256 ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.52865677,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.69628953,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.52690925,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.56758758,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.55939936,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.71661278,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.63338788,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.54263606,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.6881212 ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.61099859,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.7

In [25]:
cfa.factor_varcovs_

array([[ 1.        ,  0.39602457,  0.68207175, -0.23527996,  0.31341197],
       [ 0.39602457,  1.        ,  0.36050909, -0.34581177,  0.25281207],
       [ 0.68207175,  0.36050909,  1.        , -0.26689753,  0.43298207],
       [-0.23527996, -0.34581177, -0.26689753,  1.        , -0.10618535],
       [ 0.31341197,  0.25281207,  0.43298207, -0.10618535,  1.        ]])

## ***Una posible mejora.***

Valor a eliminar la variable con menos peso, es decir "O4"

In [26]:
## df temporal
tmp = X_CFA.drop(columns=["O4"])
scaler_X_CFA = StandardScaler()
scaler_X_CFA.fit(tmp)
scaled_data_X_CFA = scaler_X_CFA.transform(tmp)

In [27]:
model_dict = {"F1":[f"A{i}" for i in range(1,6)], "F2":[f"C{i}" for i in range(1,6)],
              "F3":[f"E{i}" for i in range(1,6)], "F4":[f"N{i}" for i in range(1,6)],
              "F5":[f"O{i}" for i in range(1,6) if i!=4]}
model_dict

{'F1': ['A1', 'A2', 'A3', 'A4', 'A5'],
 'F2': ['C1', 'C2', 'C3', 'C4', 'C5'],
 'F3': ['E1', 'E2', 'E3', 'E4', 'E5'],
 'F4': ['N1', 'N2', 'N3', 'N4', 'N5'],
 'F5': ['O1', 'O2', 'O3', 'O5']}

In [28]:
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(scaled_data_X_CFA, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp = True)
cfa.fit(scaled_data_X_CFA)

ConfirmatoryFactorAnalyzer(n_obs=1218,
                           specification=<factor_analyzer.confirmatory_factor_analyzer.ModelSpecification object at 0x786d3fdd38e0>)

In [29]:
get_statistics_cfa(cfa)

,GFI,AGFI,RMSRA,BIC,AIC
value,0.860556,0.827135,0.074688,77455.336406,76521.127729


***Note que las estadísticas revisadas reciben una mejora.***

## ***Referencias.***

1. _An Introduction to Applied Multivariate Analysis with R. Everitt B. & Hothorn T. Springer. First edition._